In [1]:
from __future__ import print_function
from __future__ import division
import os, sys
import numpy as np
import pandas as pd
from scipy.signal import medfilt
from builtins import range
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
import mir_eval
import librosa, librosa.display
import matplotlib.pyplot as plt
% matplotlib inline

import tensorflow as tf
import keras
from keras.models import Sequential, Model
from keras.layers import Activation, Dense, Flatten, Input, Reshape, Dropout, Permute
from keras.layers.convolutional import Conv2D
from keras.layers.normalization import BatchNormalization
from keras.layers.recurrent import GRU
from keras.layers.pooling import MaxPooling2D, GlobalAveragePooling2D
from keras.layers.merge import Concatenate
from keras import backend as K
from keras.backend.tensorflow_backend import set_session
from keras.layers import concatenate as concat

os.environ["CUDA_VISIBLE_DEVICES"]="0" # the number of the GPU
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.1 # percentage to be used
set_session(tf.Session(config=config))

from kapre.time_frequency import Melspectrogram
from global_config import *


vd_pedal_onset = os.path.join(DIR_PEDAL_METADATA, 'pedal-onset_vd.csv')
vd_pedal_segment = os.path.join(DIR_PEDAL_METADATA, 'pedal-segment_vd.csv')
DF_ONSET = pd.read_csv(vd_pedal_onset)
DF_SEGMENT = pd.read_csv(vd_pedal_segment)

Using TensorFlow backend.


Channel-last, i.e., (None, n_freq, n_time, n_ch)


In [3]:
DF_ONSET_20024689 = DF_ONSET.loc[DF_ONSET['category'] != 'test']
DF_SEGMENT_20024689 = DF_SEGMENT.loc[DF_SEGMENT['category'] != 'test']
subexcerpts_num = 1000

txt_filename = 'filename-20024689-composer.txt'
txt_path = os.path.join(DIR_PEDAL_METADATA, txt_filename)
filenames = np.genfromtxt(txt_path, dtype=None)

list_with_keys, list_with_values = [], []
for filename in filenames:
    filename_list = filename.split("/")
    list_with_keys.append("{}/{}".format(filename_list[0],filename_list[1])) 
    list_with_values.append(filename_list[2])

filename_composer = dict(zip(list_with_keys, list_with_values))

author = []
for filename in DF_ONSET_20024689.filename.values:
    filename_list = filename.split("-")
    key_str = "{}/{}".format(filename_list[0],filename_list[1])
    value_str = filename_composer[key_str]
    author.append(value_str)
DF_ONSET_20024689 = DF_ONSET_20024689.copy()
DF_ONSET_20024689['author'] = author

author = []
for filename in DF_SEGMENT_20024689.filename.values:
    filename_list = filename.split("-")
    key_str = "{}/{}".format(filename_list[0],filename_list[1])
    value_str = filename_composer[key_str]
    author.append(value_str) 
DF_SEGMENT_20024689 = DF_SEGMENT_20024689.copy()
DF_SEGMENT_20024689['author'] = author

unique_author = np.unique(DF_SEGMENT_20024689.author.values)

onset_20024689_author_counter = dict()
for authorname in unique_author:
    onset_20024689_author_counter[authorname] = len(DF_ONSET_20024689.loc[DF_ONSET_20024689['author'] == authorname])

segment_20024689_author_counter = dict()
for authorname in unique_author:
    segment_20024689_author_counter[authorname] = len(DF_SEGMENT_20024689.loc[DF_SEGMENT_20024689['author'] == authorname])

onset_20024689_author_subnum = dict()
for k, v in onset_20024689_author_counter.iteritems():
    if v<subexcerpts_num:
        onset_20024689_author_subnum[k] = v
    else:
        onset_20024689_author_subnum[k] = subexcerpts_num
onset_20024689_author_subnum_sum = np.sum(onset_20024689_author_subnum.values())   

segment_20024689_author_subnum = dict()
for k, v in segment_20024689_author_counter.iteritems():
    if v<subexcerpts_num:
        segment_20024689_author_subnum[k] = v
    else:
        segment_20024689_author_subnum[k] = subexcerpts_num
segment_20024689_author_subnum_sum = np.sum(segment_20024689_author_subnum.values())   

Split from `onset_20024689_author_counter` and `segment_20024689_author_counter` to train/valid set as 0.8/0.2 (composer wise).

In [7]:
import random

onset_tv_df = pd.DataFrame()
for authorname, num in onset_20024689_author_counter.iteritems():
    onset_sub = DF_ONSET_20024689.loc[DF_ONSET_20024689['author'] == authorname]
    pedal_indx = np.arange(0, len(onset_sub)-1, 2, dtype=int)
    pedal_indx_num = random.sample(pedal_indx, int(num/2))
    pedal_indx_num_train = random.sample(pedal_indx_num, int(len(pedal_indx_num)*0.8))
    pedal_indx_num_valid = [e for e in pedal_indx_num if e not in pedal_indx_num_train]
    indx_num_train = np.sort(np.concatenate((np.asarray(pedal_indx_num_train),np.asarray(pedal_indx_num_train)+1), axis=0))
    indx_num_valid = np.sort(np.concatenate((np.asarray(pedal_indx_num_valid),np.asarray(pedal_indx_num_valid)+1), axis=0))

    onset_sub_train = onset_sub.iloc[indx_num_train]
    onset_sub_valid = onset_sub.iloc[indx_num_valid]
    onset_sub_train.loc[:,'category'] = 'train'
    onset_sub_valid.loc[:,'category'] = 'valid'
    onset_sub_tv = onset_sub_train.append(onset_sub_valid)

    if len(onset_tv_df) == 0:
        onset_tv_df = onset_sub_tv.copy()
    else:
        onset_tv_df = onset_tv_df.append(onset_sub_tv)

onset_tv_df.to_csv(os.path.join(DIR_PEDAL_METADATA, 'pedal-onset_tvdf.csv'))

segment_tv_df = pd.DataFrame()
for authorname, num in segment_20024689_author_counter.iteritems():
    segment_sub = DF_SEGMENT_20024689.loc[DF_SEGMENT_20024689['author'] == authorname]
    pedal_indx = np.arange(0, len(segment_sub)-1, 2, dtype=int)
    pedal_indx_num = random.sample(pedal_indx, int(num/2))
    pedal_indx_num_train = random.sample(pedal_indx_num, int(len(pedal_indx_num)*0.8))
    pedal_indx_num_valid = [e for e in pedal_indx_num if e not in pedal_indx_num_train]
    indx_num_train = np.sort(np.concatenate((np.asarray(pedal_indx_num_train),np.asarray(pedal_indx_num_train)+1), axis=0))
    indx_num_valid = np.sort(np.concatenate((np.asarray(pedal_indx_num_valid),np.asarray(pedal_indx_num_valid)+1), axis=0))

    segment_sub_train = segment_sub.iloc[indx_num_train]
    segment_sub_valid = segment_sub.iloc[indx_num_valid]
    segment_sub_train.loc[:,'category'] = 'train'
    segment_sub_valid.loc[:,'category'] = 'valid'
    segment_sub_tv = segment_sub_train.append(segment_sub_valid)

    if len(segment_tv_df) == 0:
        segment_tv_df = segment_sub_tv.copy()
    else:
        segment_tv_df = segment_tv_df.append(segment_sub_tv)

segment_tv_df.to_csv(os.path.join(DIR_PEDAL_METADATA, 'pedal-segment_tvdf.csv'))

/homes/bl301/tf-env/lib/python2.7/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [9]:
print("======== excerpts from 20024689 ========")
print("    excerpts composers: {}".format(len(np.unique(segment_20024689_author_counter.keys()))))
print("  onset excerpts total: {}".format(np.sum(onset_20024689_author_counter.values())))
print("segment excerpts total: {}".format(np.sum(segment_20024689_author_counter.values())))

print("============= sub excerpts =============")
print("  onset sub-excerpts/%: {}/{:.2f}".format(onset_20024689_author_subnum_sum,
                                                 onset_20024689_author_subnum_sum/np.sum(onset_20024689_author_counter.values())))
print("segment sub-excerpts/%: {}/{:.2f}".format(segment_20024689_author_subnum_sum,
                                                 segment_20024689_author_subnum_sum/np.sum(segment_20024689_author_counter.values())))

======== excerpts from 20024689 ========
    excerpts composers: 84
  onset excerpts total: 1134716
segment excerpts total: 903398
============= sub excerpts =============
  onset sub-excerpts/%: 67540/0.06
segment sub-excerpts/%: 62424/0.07


## Make sub datasets

Get excerpts within sub synth datasets of year 20024689 from every composers to form the small version dataset. Train/Valid set is split as 0.8/0.2 from every composer-wise excerpts.

Make the sub datasets 10 times for checking the conficence interval.

In [4]:
import random

confidence_interval_times = 10

onset_sub_df = pd.DataFrame()
for interval_time in range(confidence_interval_times):
    print("{} ...".format(interval_time))
    for authorname, num in onset_20024689_author_subnum.iteritems():
        onset_sub = DF_ONSET_20024689.loc[DF_ONSET_20024689['author'] == authorname]
        pedal_indx = np.arange(0, len(onset_sub)-1, 2, dtype=int)
        pedal_indx_num = random.sample(pedal_indx, int(num/2))
        pedal_indx_num_train = random.sample(pedal_indx_num, int(len(pedal_indx_num)*0.8))
        pedal_indx_num_valid = [e for e in pedal_indx_num if e not in pedal_indx_num_train]
        indx_num_train = np.sort(np.concatenate((np.asarray(pedal_indx_num_train),np.asarray(pedal_indx_num_train)+1), axis=0))
        indx_num_valid = np.sort(np.concatenate((np.asarray(pedal_indx_num_valid),np.asarray(pedal_indx_num_valid)+1), axis=0))

        onset_sub_train = onset_sub.iloc[indx_num_train]
        onset_sub_valid = onset_sub.iloc[indx_num_valid]
        onset_sub_train.loc[:,'category'] = 'train'
        onset_sub_valid.loc[:,'category'] = 'valid'
        onset_sub_tv = onset_sub_train.append(onset_sub_valid)
        onset_sub_tv = onset_sub_tv.copy()
        onset_sub_tv['interval_time'] = [interval_time] * len(onset_sub_tv) 
        
        if len(onset_sub_df) == 0:
            onset_sub_df = onset_sub_tv.copy()
        else:
            onset_sub_df = onset_sub_df.append(onset_sub_tv)
        
    print("  done!")

onset_sub_df.to_csv(os.path.join(DIR_PEDAL_METADATA, 'pedal-onset_subdf.csv'))

segment_sub_df = pd.DataFrame()
for interval_time in range(confidence_interval_times):
    print("{} ...".format(interval_time))
    for authorname, num in segment_20024689_author_subnum.iteritems():
        segment_sub = DF_SEGMENT_20024689.loc[DF_SEGMENT_20024689['author'] == authorname]
        pedal_indx = np.arange(0, len(segment_sub)-1, 2, dtype=int)
        pedal_indx_num = random.sample(pedal_indx, int(num/2))
        pedal_indx_num_train = random.sample(pedal_indx_num, int(len(pedal_indx_num)*0.8))
        pedal_indx_num_valid = [e for e in pedal_indx_num if e not in pedal_indx_num_train]
        indx_num_train = np.sort(np.concatenate((np.asarray(pedal_indx_num_train),np.asarray(pedal_indx_num_train)+1), axis=0))
        indx_num_valid = np.sort(np.concatenate((np.asarray(pedal_indx_num_valid),np.asarray(pedal_indx_num_valid)+1), axis=0))

        segment_sub_train = segment_sub.iloc[indx_num_train]
        segment_sub_valid = segment_sub.iloc[indx_num_valid]
        segment_sub_train.loc[:,'category'] = 'train'
        segment_sub_valid.loc[:,'category'] = 'valid'
        segment_sub_tv = segment_sub_train.append(segment_sub_valid)
        segment_sub_tv = segment_sub_tv.copy()
        segment_sub_tv['interval_time'] = [interval_time] * len(segment_sub_tv) 
        
        if len(segment_sub_df) == 0:
            segment_sub_df = segment_sub_tv.copy()
        else:
            segment_sub_df = segment_sub_df.append(segment_sub_tv)
            
    print("  done!")

segment_sub_df.to_csv(os.path.join(DIR_PEDAL_METADATA, 'pedal-segment_subdf.csv'))

0 ...


/homes/bl301/tf-env/lib/python2.7/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


  done!
1 ...
  done!
2 ...
  done!
3 ...
  done!
4 ...
  done!
5 ...
  done!
6 ...
  done!
7 ...
  done!
8 ...
  done!
9 ...
  done!
0 ...
  done!
1 ...
  done!
2 ...
  done!
3 ...
  done!
4 ...
  done!
5 ...
  done!
6 ...
  done!
7 ...
  done!
8 ...
  done!
9 ...
  done!


In [8]:
print("=== sub excerpts ===")
print("  onset train: {}".format(len(onset_sub_df.loc[(onset_sub_df['category'] == 'train') & (onset_sub_df['interval_time'] == 0)])))
print("  onset valid: {}".format(len(onset_sub_df.loc[(onset_sub_df['category'] == 'valid') & (onset_sub_df['interval_time'] == 0)])))
print("segment train: {}".format(len(segment_sub_df.loc[(segment_sub_df['category'] == 'train') & (segment_sub_df['interval_time'] == 0)])))
print("segment valid: {}".format(len(segment_sub_df.loc[(segment_sub_df['category'] == 'valid') & (segment_sub_df['interval_time'] == 0)])))

=== sub excerpts ===
  onset train: 54004
  onset valid: 13536
segment train: 49908
segment valid: 12516


## MAESTRO Dataset

We will use pieces from year 2013457 as part of the real audio dataset. Train/Valid/Test split follows the maestro setting.

In [16]:
MAESTRO_DF = pd.read_csv('/import/c4dm-05/bl301/sustain-pedal-detection/maestro-v1.0.0/maestro-v1.0.0.csv')

maestro_df = MAESTRO_DF.loc[MAESTRO_DF['year'] > 2011]

author = []
for composer in maestro_df.canonical_composer.values:
    composer_list = composer.split(" ")
    if "/" in composer_list:
        surname = composer_list[composer_list.index("/")-1]
    else:
        surname = composer_list[-1]
    
    if surname == 'Jan\xc3\xa1\xc4\x8dek':
        surname = 'Janacek'
    elif surname == 'Rachmaninoff':
        surname = 'Rachmaninov'
    author.append(surname) 
maestro_df = maestro_df.copy()
maestro_df['author'] = author

print("  maestro composers: {}".format(len(np.unique(maestro_df.author.values))))
print("maestro piece total: {}".format(len(maestro_df)))
print("              train: {}".format(len(maestro_df.loc[MAESTRO_DF['split']=='train'])))
print("              valid: {}".format(len(maestro_df.loc[MAESTRO_DF['split']=='validation'])))
print("               test: {}".format(len(maestro_df.loc[MAESTRO_DF['split']=='test'])))
# maestro_df.to_csv(os.path.join(DIR_PEDAL_METADATA, 'pedal-maestro_2013457df.csv'))

  maestro composers: 25
maestro piece total: 502
              train: 404
              valid: 48
               test: 50


We get the pedal onset and offset times from midi data, and save them into `pedal-times_maestro.npz` with other meta data.

In [41]:
import pretty_midi
filenames, categories, authors= [], [], []
pedal_onsets, pedal_offsets = [], []
filename_list = maestro_df.audio_filename.values
midiname_list = maestro_df.midi_filename.values
category_list = maestro_df.split.values
author_list = maestro_df.author.values
MAESTRO_PATH = '/import/c4dm-05/bl301/sustain-pedal-detection/maestro-v1.0.0'
for indx,filename in enumerate(filename_list):  
    midi_path = os.path.join(MAESTRO_PATH, midiname_list[indx])

    # get ground truth pedal onset time from midi
    pm = pretty_midi.PrettyMIDI(midi_path)
    pedal_v = []
    pedal_t = []
    for control_change in pm.instruments[0].control_changes:
        if control_change.number == 64:
            pedal_v.append(control_change.value)
            pedal_t.append(control_change.time)

    pedal_onset = []
    pedal_offset = []
    for i,v in enumerate(pedal_v):
        if i>0 and v>=64 and pedal_v[i-1]<64:
            pedal_onset.append(pedal_t[i])   
        elif i>0 and v<64 and pedal_v[i-1]>=64:
            pedal_offset.append(pedal_t[i])

    pedal_offset = [t for t in pedal_offset if t > pedal_onset[0]]
    seg_idxs = np.min([len(pedal_onset), len(pedal_offset)])
    pedal_offset = pedal_offset[:seg_idxs]
    pedal_onset = pedal_onset[:seg_idxs]
    for seg_idx, offset in enumerate(pedal_offset):
        if offset != pedal_offset[-1] and offset > pedal_onset[seg_idx] and offset < pedal_onset[seg_idx+1]:
            correct_pedal_data = True
        elif offset == pedal_offset[-1] and offset > pedal_onset[seg_idx]:
            correct_pedal_data = True
        else:
            correct_pedal_data = False

    if correct_pedal_data:
        filenames.append(filename)
        pedal_onsets.append(pedal_onset)
        pedal_offsets.append(pedal_offset)
        categories.append(category_list[indx])
        authors.append(author_list[indx])

np.savez(os.path.join(DIR_PEDAL_METADATA, 'pedal-times_maestro.npz'), 
         filename=filenames, pedal_onset=pedal_onsets, pedal_offset=pedal_offsets, 
         category=categories, author=authors)
print('pedal-times_maestro.npz is saved!')

pedal-times_maestro.npz is saved!


In [53]:
npz_path = os.path.join(DIR_PEDAL_METADATA, 'pedal-times_maestro.npz')
maestro_tracks = np.load(npz_path)

maestro_train_indx = np.where(maestro_tracks['category'] == 'train')[0]
maestro_valid_indx = np.where(maestro_tracks['category'] == 'validation')[0]
maestro_test_indx = np.where(maestro_tracks['category'] == 'test')[0]